In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pymc3 import all_continuous

AttributeError: module 'arviz' has no attribute 'geweke'

In [46]:
county_data = pd.read_csv(os.path.realpath("./Data/us_county.csv"))
covid_county_data = pd.read_csv(os.path.realpath("./Data/covid_us_county.csv")).filter(items=["county","state_code","lat","long",
                                                                                                 "date","cases","deaths"])

In [48]:
#filter covid data on observations > 0
covid_county_data = covid_county_data[covid_county_data['cases'] > 0]
covid_county_data.head(5)

,county,state_code,lat,long,date,cases,deaths
62,Autauga,AL,32.539527,-86.644082,2020-03-24,1,0.0
63,Autauga,AL,32.539527,-86.644082,2020-03-25,5,0.0
64,Autauga,AL,32.539527,-86.644082,2020-03-26,6,0.0
65,Autauga,AL,32.539527,-86.644082,2020-03-27,6,0.0
66,Autauga,AL,32.539527,-86.644082,2020-03-28,6,0.0


In [51]:
first_dates_by_county = covid_county_data.filter(items=['county','state_code','date']).groupby(['county','state_code']).first().sort_values(by='date')
first_dates_by_county[0:10]

,,date
county,state_code,
King,WA,2020-01-22
Cook,IL,2020-01-24
Orange,CA,2020-01-26
Maricopa,AZ,2020-01-26
Los Angeles,CA,2020-01-26
Suffolk,MA,2020-01-29
Santa Clara,CA,2020-01-31
San Benito,CA,2020-02-03
Worcester,MA,2020-02-06


# What are the generations for this model?
   - To discretize this, we are looking at snapshots in time of a COVID propagation in certain areas
     - X(0) would be first time window. X(1) would be second and so on...
     - Perhaps we model this in increments of every 5 days (case symptom onset)
   - To generate a model for the probability 1 person spreads COVID to k = 0,1,2,... we could use a $X|N \in Bin(n,p)$ with $N \in Po(\lambda)$
      - Where X is the random variable representing the probability that a person passes on COVID to k = 0,1,2,.. people
      - We can generate the probability using transforms (pg. 78 GIT) 
      - $\lambda$ is the random variable representing the distribution of people a person comes in contact with during a "generation"
   - Can we attempt to use the data obtained to find a distribution for the parameters of the model?
      - $Pr(\lambda,p|X=difference in case counts by day)$